In [1]:
#importing the libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sb

from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
#reading the train(featrue encoded) csv file
train_df = pd.read_csv("../data/train_encoded.csv")

In [3]:
#shape of data
train_df.shape

(614, 13)

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Loan_ID           614 non-null    object 
 1   Gender            601 non-null    float64
 2   Married           611 non-null    float64
 3   Dependents        599 non-null    float64
 4   Education         614 non-null    int64  
 5   Self_Employed     582 non-null    float64
 6   LoanAmount        592 non-null    float64
 7   Loan_Amount_Term  600 non-null    float64
 8   Credit_History    564 non-null    float64
 9   Loan_Status       614 non-null    int64  
 10  Total_Income      614 non-null    float64
 11  Semiurban         614 non-null    int64  
 12  Urban             614 non-null    int64  
dtypes: float64(8), int64(4), object(1)
memory usage: 62.5+ KB


In [5]:
#looking at the top 5 records
train_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Total_Income,Semiurban,Urban
0,LP001002,1.0,0.0,0.0,1,0.0,NaN,360.0,1.0,1,5849.0,0,1
1,LP001003,1.0,1.0,1.0,1,0.0,128.0,360.0,1.0,0,6091.0,0,0
2,LP001005,1.0,1.0,0.0,1,1.0,66.0,360.0,1.0,1,3000.0,0,1
3,LP001006,1.0,1.0,0.0,0,0.0,120.0,360.0,1.0,1,4941.0,0,1
4,LP001008,1.0,0.0,0.0,1,0.0,141.0,360.0,1.0,1,6000.0,0,1


In [6]:
def check_null(df):
    '''
    Function returns % of null values in each column
    
    Args:
        df(Pandas DataFrame)
        
    Returns:
        Pandas Series: columns as index and the percentage of null values as value
    '''
    nulls = (df.isnull().sum())/len(df)
    return 100*nulls.sort_values(ascending=False)

In [7]:
check_null(train_df)

Credit_History      8.143322
Self_Employed       5.211726
LoanAmount          3.583062
Dependents          2.442997
Loan_Amount_Term    2.280130
Gender              2.117264
Married             0.488599
Loan_ID             0.000000
Education           0.000000
Loan_Status         0.000000
Total_Income        0.000000
Semiurban           0.000000
Urban               0.000000
dtype: float64

## Imputation

In [8]:
#importing the libraries for imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

In [9]:
X_train = train_df.drop(['Loan_ID', 'Loan_Status'], axis=1)
y_train = train_df['Loan_Status']

In [10]:
itr_imputer = IterativeImputer()

In [11]:
knn_imputer = KNNImputer()

In [12]:
import time

In [13]:
start = time.time()
itr_imputed = itr_imputer.fit_transform(X_train,y_train)
print(f"Time taken: {(time.time() - start)/60} minutes.")

Time taken: 0.004277825355529785 minutes.


In [14]:
start = time.time()
knn_imputed = knn_imputer.fit_transform(X_train,y_train)
print(f"Time taken: {(time.time() - start)/60} minutes.")

Time taken: 0.0007646322250366211 minutes.


In [15]:
itr_imputed.shape, knn_imputed.shape

((614, 11), (614, 11))

#### KNN imputer is 10 times faster than Iterative Imputer

## Scaling the data

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
scaler = StandardScaler()

In [18]:
itr_imputed_scaled = scaler.fit_transform(itr_imputed)

In [19]:
knn_imputed_scaled = scaler.fit_transform(knn_imputed)

### Comparing the imputations

In [20]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

#### Iterative Imputer Score

In [21]:
lr_score = cross_val_score(LogisticRegression(), X = itr_imputed_scaled, y=y_train, scoring="accuracy", cv=5)
svc_score = cross_val_score(SVC(), X = itr_imputed_scaled, y=y_train, scoring="accuracy", cv=5)
rf_score = cross_val_score(RandomForestClassifier(), X = itr_imputed, y=y_train, scoring="accuracy", cv=5)
ada_score = cross_val_score(AdaBoostClassifier(), X = itr_imputed, y=y_train, scoring="accuracy", cv=5)
xgb_score = cross_val_score(XGBClassifier(use_label_encoder=False), X = itr_imputed, y=y_train, scoring="accuracy", cv=5)
cat_score = cross_val_score(CatBoostClassifier(), X = itr_imputed, y=y_train, scoring="accuracy", cv=5)

[15:31:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:31:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:31:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:31:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

175:	learn: 0.4322938	total: 1.23s	remaining: 5.77s
176:	learn: 0.4317369	total: 1.24s	remaining: 5.75s
177:	learn: 0.4310955	total: 1.24s	remaining: 5.72s
178:	learn: 0.4304928	total: 1.24s	remaining: 5.69s
179:	learn: 0.4299459	total: 1.24s	remaining: 5.67s
180:	learn: 0.4293358	total: 1.25s	remaining: 5.64s
181:	learn: 0.4289084	total: 1.25s	remaining: 5.62s
182:	learn: 0.4282958	total: 1.25s	remaining: 5.59s
183:	learn: 0.4278264	total: 1.25s	remaining: 5.57s
184:	learn: 0.4275559	total: 1.26s	remaining: 5.54s
185:	learn: 0.4271476	total: 1.26s	remaining: 5.52s
186:	learn: 0.4268898	total: 1.26s	remaining: 5.5s
187:	learn: 0.4264378	total: 1.27s	remaining: 5.47s
188:	learn: 0.4259072	total: 1.27s	remaining: 5.45s
189:	learn: 0.4252955	total: 1.27s	remaining: 5.42s
190:	learn: 0.4250073	total: 1.27s	remaining: 5.4s
191:	learn: 0.4245983	total: 1.28s	remaining: 5.38s
192:	learn: 0.4240541	total: 1.28s	remaining: 5.36s
193:	learn: 0.4235621	total: 1.28s	remaining: 5.34s
194:	learn: 0.

358:	learn: 0.3656176	total: 1.75s	remaining: 3.12s
359:	learn: 0.3654340	total: 1.75s	remaining: 3.11s
360:	learn: 0.3652549	total: 1.75s	remaining: 3.1s
361:	learn: 0.3649263	total: 1.75s	remaining: 3.09s
362:	learn: 0.3647157	total: 1.76s	remaining: 3.08s
363:	learn: 0.3644133	total: 1.76s	remaining: 3.08s
364:	learn: 0.3640822	total: 1.76s	remaining: 3.07s
365:	learn: 0.3637207	total: 1.77s	remaining: 3.06s
366:	learn: 0.3634752	total: 1.77s	remaining: 3.05s
367:	learn: 0.3630264	total: 1.77s	remaining: 3.04s
368:	learn: 0.3628356	total: 1.77s	remaining: 3.03s
369:	learn: 0.3627663	total: 1.77s	remaining: 3.02s
370:	learn: 0.3626318	total: 1.78s	remaining: 3.01s
371:	learn: 0.3624901	total: 1.78s	remaining: 3s
372:	learn: 0.3621977	total: 1.78s	remaining: 3s
373:	learn: 0.3620100	total: 1.78s	remaining: 2.99s
374:	learn: 0.3616181	total: 1.79s	remaining: 2.98s
375:	learn: 0.3614134	total: 1.79s	remaining: 2.97s
376:	learn: 0.3612121	total: 1.79s	remaining: 2.96s
377:	learn: 0.36084

536:	learn: 0.3228566	total: 2.27s	remaining: 1.96s
537:	learn: 0.3226251	total: 2.27s	remaining: 1.95s
538:	learn: 0.3224077	total: 2.27s	remaining: 1.95s
539:	learn: 0.3222056	total: 2.28s	remaining: 1.94s
540:	learn: 0.3220183	total: 2.28s	remaining: 1.94s
541:	learn: 0.3218157	total: 2.29s	remaining: 1.93s
542:	learn: 0.3216443	total: 2.29s	remaining: 1.93s
543:	learn: 0.3212439	total: 2.29s	remaining: 1.92s
544:	learn: 0.3208513	total: 2.3s	remaining: 1.92s
545:	learn: 0.3204022	total: 2.3s	remaining: 1.91s
546:	learn: 0.3202466	total: 2.3s	remaining: 1.91s
547:	learn: 0.3200739	total: 2.31s	remaining: 1.9s
548:	learn: 0.3198071	total: 2.31s	remaining: 1.9s
549:	learn: 0.3196017	total: 2.31s	remaining: 1.89s
550:	learn: 0.3194580	total: 2.32s	remaining: 1.89s
551:	learn: 0.3191496	total: 2.32s	remaining: 1.88s
552:	learn: 0.3189063	total: 2.32s	remaining: 1.88s
553:	learn: 0.3186614	total: 2.33s	remaining: 1.87s
554:	learn: 0.3184384	total: 2.33s	remaining: 1.87s
555:	learn: 0.318

753:	learn: 0.2806916	total: 2.96s	remaining: 965ms
754:	learn: 0.2805090	total: 2.96s	remaining: 961ms
755:	learn: 0.2801894	total: 2.96s	remaining: 957ms
756:	learn: 0.2800173	total: 2.97s	remaining: 953ms
757:	learn: 0.2798596	total: 2.97s	remaining: 948ms
758:	learn: 0.2795942	total: 2.97s	remaining: 944ms
759:	learn: 0.2793360	total: 2.98s	remaining: 940ms
760:	learn: 0.2791496	total: 2.98s	remaining: 936ms
761:	learn: 0.2789804	total: 2.98s	remaining: 932ms
762:	learn: 0.2788664	total: 2.98s	remaining: 927ms
763:	learn: 0.2785818	total: 2.99s	remaining: 923ms
764:	learn: 0.2784462	total: 2.99s	remaining: 919ms
765:	learn: 0.2782336	total: 3s	remaining: 915ms
766:	learn: 0.2781028	total: 3s	remaining: 911ms
767:	learn: 0.2778518	total: 3s	remaining: 906ms
768:	learn: 0.2775775	total: 3s	remaining: 902ms
769:	learn: 0.2774278	total: 3.01s	remaining: 898ms
770:	learn: 0.2772342	total: 3.01s	remaining: 894ms
771:	learn: 0.2769671	total: 3.01s	remaining: 890ms
772:	learn: 0.2767873	to

...

274:	learn: 0.3838478	total: 882ms	remaining: 2.33s
275:	learn: 0.3837400	total: 886ms	remaining: 2.32s
276:	learn: 0.3833305	total: 890ms	remaining: 2.32s
277:	learn: 0.3828631	total: 894ms	remaining: 2.32s
278:	learn: 0.3824832	total: 898ms	remaining: 2.32s
279:	learn: 0.3821370	total: 902ms	remaining: 2.32s
280:	learn: 0.3817429	total: 907ms	remaining: 2.32s
281:	learn: 0.3815141	total: 911ms	remaining: 2.32s
282:	learn: 0.3810616	total: 915ms	remaining: 2.32s
283:	learn: 0.3806542	total: 919ms	remaining: 2.32s
284:	learn: 0.3804131	total: 924ms	remaining: 2.32s
285:	learn: 0.3801493	total: 928ms	remaining: 2.32s
286:	learn: 0.3798664	total: 932ms	remaining: 2.31s
287:	learn: 0.3795676	total: 936ms	remaining: 2.31s
288:	learn: 0.3790747	total: 941ms	remaining: 2.31s
289:	learn: 0.3788227	total: 945ms	remaining: 2.31s
290:	learn: 0.3784602	total: 949ms	remaining: 2.31s
291:	learn: 0.3782556	total: 953ms	remaining: 2.31s
292:	learn: 0.3779370	total: 957ms	remaining: 2.31s
293:	learn: 

447:	learn: 0.3348803	total: 1.76s	remaining: 2.16s
448:	learn: 0.3344880	total: 1.76s	remaining: 2.16s
449:	learn: 0.3342912	total: 1.77s	remaining: 2.16s
450:	learn: 0.3339479	total: 1.77s	remaining: 2.16s
451:	learn: 0.3335902	total: 1.78s	remaining: 2.16s
452:	learn: 0.3333026	total: 1.78s	remaining: 2.15s
453:	learn: 0.3331887	total: 1.79s	remaining: 2.15s
454:	learn: 0.3329303	total: 1.79s	remaining: 2.15s
455:	learn: 0.3327217	total: 1.8s	remaining: 2.15s
456:	learn: 0.3325256	total: 1.81s	remaining: 2.15s
457:	learn: 0.3323214	total: 1.81s	remaining: 2.14s
458:	learn: 0.3321241	total: 1.82s	remaining: 2.14s
459:	learn: 0.3320257	total: 1.82s	remaining: 2.14s
460:	learn: 0.3317541	total: 1.83s	remaining: 2.14s
461:	learn: 0.3315464	total: 1.83s	remaining: 2.13s
462:	learn: 0.3312113	total: 1.84s	remaining: 2.13s
463:	learn: 0.3310618	total: 1.84s	remaining: 2.13s
464:	learn: 0.3308894	total: 1.85s	remaining: 2.13s
465:	learn: 0.3305871	total: 1.85s	remaining: 2.13s
466:	learn: 0

633:	learn: 0.2982512	total: 2.75s	remaining: 1.59s
634:	learn: 0.2980739	total: 2.76s	remaining: 1.59s
635:	learn: 0.2979397	total: 2.76s	remaining: 1.58s
636:	learn: 0.2977573	total: 2.77s	remaining: 1.58s
637:	learn: 0.2975127	total: 2.77s	remaining: 1.57s
638:	learn: 0.2973720	total: 2.78s	remaining: 1.57s
639:	learn: 0.2971784	total: 2.79s	remaining: 1.57s
640:	learn: 0.2970760	total: 2.79s	remaining: 1.56s
641:	learn: 0.2968901	total: 2.8s	remaining: 1.56s
642:	learn: 0.2967875	total: 2.8s	remaining: 1.55s
643:	learn: 0.2966224	total: 2.81s	remaining: 1.55s
644:	learn: 0.2964736	total: 2.81s	remaining: 1.55s
645:	learn: 0.2963229	total: 2.82s	remaining: 1.54s
646:	learn: 0.2961767	total: 2.82s	remaining: 1.54s
647:	learn: 0.2960472	total: 2.83s	remaining: 1.54s
648:	learn: 0.2958150	total: 2.83s	remaining: 1.53s
649:	learn: 0.2953960	total: 2.84s	remaining: 1.53s
650:	learn: 0.2952194	total: 2.85s	remaining: 1.52s
651:	learn: 0.2949839	total: 2.85s	remaining: 1.52s
652:	learn: 0.

812:	learn: 0.2676383	total: 3.75s	remaining: 862ms
813:	learn: 0.2674610	total: 3.75s	remaining: 858ms
814:	learn: 0.2672235	total: 3.76s	remaining: 854ms
815:	learn: 0.2670241	total: 3.77s	remaining: 849ms
816:	learn: 0.2669377	total: 3.77s	remaining: 845ms
817:	learn: 0.2667330	total: 3.78s	remaining: 841ms
818:	learn: 0.2666269	total: 3.78s	remaining: 836ms
819:	learn: 0.2665827	total: 3.79s	remaining: 832ms
820:	learn: 0.2664584	total: 3.8s	remaining: 828ms
821:	learn: 0.2663483	total: 3.8s	remaining: 823ms
822:	learn: 0.2662588	total: 3.81s	remaining: 819ms
823:	learn: 0.2661461	total: 3.81s	remaining: 815ms
824:	learn: 0.2658647	total: 3.82s	remaining: 811ms
825:	learn: 0.2658228	total: 3.83s	remaining: 806ms
826:	learn: 0.2656332	total: 3.83s	remaining: 802ms
827:	learn: 0.2655388	total: 3.84s	remaining: 798ms
828:	learn: 0.2653220	total: 3.85s	remaining: 793ms
829:	learn: 0.2651967	total: 3.85s	remaining: 789ms
830:	learn: 0.2649295	total: 3.86s	remaining: 785ms
831:	learn: 0.

976:	learn: 0.2428254	total: 4.74s	remaining: 112ms
977:	learn: 0.2427418	total: 4.75s	remaining: 107ms
978:	learn: 0.2427006	total: 4.75s	remaining: 102ms
979:	learn: 0.2424162	total: 4.76s	remaining: 97.2ms
980:	learn: 0.2422995	total: 4.77s	remaining: 92.4ms
981:	learn: 0.2420725	total: 4.78s	remaining: 87.5ms
982:	learn: 0.2419358	total: 4.78s	remaining: 82.7ms
983:	learn: 0.2418045	total: 4.79s	remaining: 77.9ms
984:	learn: 0.2416097	total: 4.79s	remaining: 73ms
985:	learn: 0.2414564	total: 4.8s	remaining: 68.2ms
986:	learn: 0.2413264	total: 4.81s	remaining: 63.3ms
987:	learn: 0.2411830	total: 4.82s	remaining: 58.5ms
988:	learn: 0.2410820	total: 4.82s	remaining: 53.6ms
989:	learn: 0.2409436	total: 4.83s	remaining: 48.8ms
990:	learn: 0.2407620	total: 4.83s	remaining: 43.9ms
991:	learn: 0.2406060	total: 4.84s	remaining: 39ms
992:	learn: 0.2405440	total: 4.85s	remaining: 34.2ms
993:	learn: 0.2404552	total: 4.85s	remaining: 29.3ms
994:	learn: 0.2403603	total: 4.86s	remaining: 24.4ms
9

In [22]:
print(f"Logistic Regression score = {lr_score.mean()}")
print(f"SVC Regression score = {svc_score.mean()}")
print(f"Random Forest Regression score = {rf_score.mean()}")
print(f"AdaBost Regression score = {ada_score.mean()}")
print(f"XGB Regression score = {xgb_score.mean()}")
print(f"CatBoost Regression score = {cat_score.mean()}")

Logistic Regression score = 0.8045715047314406
SVC Regression score = 0.8013194722111155
Random Forest Regression score = 0.7915767026522724
AdaBost Regression score = 0.7785019325603093
XGB Regression score = 0.7491536718645875
CatBoost Regression score = 0.8111022257763562


#### KNN imputer Score

In [23]:
lr_score = cross_val_score(LogisticRegression(), X = knn_imputed_scaled, y=y_train, scoring="accuracy", cv=5)
svc_score = cross_val_score(SVC(), X = knn_imputed_scaled, y=y_train, scoring="accuracy", cv=5)
rf_score = cross_val_score(RandomForestClassifier(), X = knn_imputed, y=y_train, scoring="accuracy", cv=5)
ada_score = cross_val_score(AdaBoostClassifier(), X = knn_imputed, y=y_train, scoring="accuracy", cv=5)
xgb_score = cross_val_score(XGBClassifier(use_label_encoder=False), X = knn_imputed, y=y_train, scoring="accuracy", cv=5)
cat_score = cross_val_score(CatBoostClassifier(), X = knn_imputed, y=y_train, scoring="accuracy", cv=5)

[15:32:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:32:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:32:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:32:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

184:	learn: 0.4334868	total: 341ms	remaining: 1.5s
185:	learn: 0.4330994	total: 343ms	remaining: 1.5s
186:	learn: 0.4326865	total: 345ms	remaining: 1.5s
187:	learn: 0.4320927	total: 347ms	remaining: 1.5s
188:	learn: 0.4315215	total: 349ms	remaining: 1.5s
189:	learn: 0.4311953	total: 351ms	remaining: 1.5s
190:	learn: 0.4309196	total: 353ms	remaining: 1.49s
191:	learn: 0.4303908	total: 355ms	remaining: 1.49s
192:	learn: 0.4303383	total: 356ms	remaining: 1.49s
193:	learn: 0.4298484	total: 358ms	remaining: 1.49s
194:	learn: 0.4291296	total: 360ms	remaining: 1.48s
195:	learn: 0.4284024	total: 362ms	remaining: 1.48s
196:	learn: 0.4280808	total: 363ms	remaining: 1.48s
197:	learn: 0.4275251	total: 366ms	remaining: 1.48s
198:	learn: 0.4270879	total: 368ms	remaining: 1.48s
199:	learn: 0.4268004	total: 369ms	remaining: 1.48s
200:	learn: 0.4263566	total: 371ms	remaining: 1.48s
201:	learn: 0.4258842	total: 373ms	remaining: 1.47s
202:	learn: 0.4255297	total: 375ms	remaining: 1.47s
203:	learn: 0.4251

357:	learn: 0.3706360	total: 687ms	remaining: 1.23s
358:	learn: 0.3703814	total: 690ms	remaining: 1.23s
359:	learn: 0.3700054	total: 692ms	remaining: 1.23s
360:	learn: 0.3698156	total: 695ms	remaining: 1.23s
361:	learn: 0.3695696	total: 697ms	remaining: 1.23s
362:	learn: 0.3690624	total: 699ms	remaining: 1.23s
363:	learn: 0.3686629	total: 701ms	remaining: 1.23s
364:	learn: 0.3684652	total: 704ms	remaining: 1.22s
365:	learn: 0.3681914	total: 706ms	remaining: 1.22s
366:	learn: 0.3679899	total: 709ms	remaining: 1.22s
367:	learn: 0.3679451	total: 710ms	remaining: 1.22s
368:	learn: 0.3676386	total: 712ms	remaining: 1.22s
369:	learn: 0.3673731	total: 715ms	remaining: 1.22s
370:	learn: 0.3671258	total: 717ms	remaining: 1.22s
371:	learn: 0.3668922	total: 719ms	remaining: 1.21s
372:	learn: 0.3667236	total: 721ms	remaining: 1.21s
373:	learn: 0.3664693	total: 723ms	remaining: 1.21s
374:	learn: 0.3660163	total: 726ms	remaining: 1.21s
375:	learn: 0.3657106	total: 728ms	remaining: 1.21s
376:	learn: 

577:	learn: 0.3182283	total: 1.19s	remaining: 867ms
578:	learn: 0.3179122	total: 1.2s	remaining: 869ms
579:	learn: 0.3177910	total: 1.2s	remaining: 869ms
580:	learn: 0.3177447	total: 1.21s	remaining: 870ms
581:	learn: 0.3175189	total: 1.21s	remaining: 872ms
582:	learn: 0.3172568	total: 1.22s	remaining: 873ms
583:	learn: 0.3170919	total: 1.23s	remaining: 874ms
584:	learn: 0.3167877	total: 1.23s	remaining: 875ms
585:	learn: 0.3165932	total: 1.24s	remaining: 876ms
586:	learn: 0.3161539	total: 1.25s	remaining: 877ms
587:	learn: 0.3159188	total: 1.25s	remaining: 879ms
588:	learn: 0.3156894	total: 1.26s	remaining: 879ms
589:	learn: 0.3153393	total: 1.27s	remaining: 880ms
590:	learn: 0.3151414	total: 1.27s	remaining: 881ms
591:	learn: 0.3150465	total: 1.28s	remaining: 882ms
592:	learn: 0.3148413	total: 1.29s	remaining: 883ms
593:	learn: 0.3145143	total: 1.29s	remaining: 883ms
594:	learn: 0.3142905	total: 1.3s	remaining: 884ms
595:	learn: 0.3141919	total: 1.3s	remaining: 885ms
596:	learn: 0.31

747:	learn: 0.2854566	total: 2.23s	remaining: 751ms
748:	learn: 0.2852668	total: 2.23s	remaining: 749ms
749:	learn: 0.2850751	total: 2.24s	remaining: 747ms
750:	learn: 0.2849770	total: 2.25s	remaining: 745ms
751:	learn: 0.2847714	total: 2.25s	remaining: 743ms
752:	learn: 0.2844113	total: 2.26s	remaining: 741ms
753:	learn: 0.2843130	total: 2.26s	remaining: 739ms
754:	learn: 0.2840991	total: 2.27s	remaining: 737ms
755:	learn: 0.2839325	total: 2.28s	remaining: 735ms
756:	learn: 0.2837564	total: 2.28s	remaining: 733ms
757:	learn: 0.2835324	total: 2.29s	remaining: 731ms
758:	learn: 0.2833307	total: 2.3s	remaining: 729ms
759:	learn: 0.2831434	total: 2.3s	remaining: 727ms
760:	learn: 0.2829161	total: 2.31s	remaining: 725ms
761:	learn: 0.2827951	total: 2.31s	remaining: 723ms
762:	learn: 0.2825082	total: 2.32s	remaining: 721ms
763:	learn: 0.2820355	total: 2.33s	remaining: 719ms
764:	learn: 0.2818092	total: 2.33s	remaining: 717ms
765:	learn: 0.2817074	total: 2.34s	remaining: 715ms
766:	learn: 0.

...

168:	learn: 0.4343882	total: 875ms	remaining: 4.3s
169:	learn: 0.4337638	total: 881ms	remaining: 4.3s
170:	learn: 0.4329086	total: 887ms	remaining: 4.3s
171:	learn: 0.4323262	total: 892ms	remaining: 4.29s
172:	learn: 0.4318486	total: 897ms	remaining: 4.29s
173:	learn: 0.4311869	total: 903ms	remaining: 4.29s
174:	learn: 0.4306762	total: 909ms	remaining: 4.28s
175:	learn: 0.4303146	total: 914ms	remaining: 4.28s
176:	learn: 0.4298683	total: 920ms	remaining: 4.28s
177:	learn: 0.4293670	total: 925ms	remaining: 4.27s
178:	learn: 0.4288471	total: 931ms	remaining: 4.27s
179:	learn: 0.4283701	total: 936ms	remaining: 4.26s
180:	learn: 0.4280124	total: 942ms	remaining: 4.26s
181:	learn: 0.4275213	total: 947ms	remaining: 4.26s
182:	learn: 0.4270245	total: 953ms	remaining: 4.25s
183:	learn: 0.4268858	total: 956ms	remaining: 4.24s
184:	learn: 0.4264588	total: 961ms	remaining: 4.24s
185:	learn: 0.4261475	total: 967ms	remaining: 4.23s
186:	learn: 0.4257265	total: 972ms	remaining: 4.22s
187:	learn: 0.4

348:	learn: 0.3652780	total: 1.87s	remaining: 3.49s
349:	learn: 0.3648812	total: 1.88s	remaining: 3.48s
350:	learn: 0.3645428	total: 1.88s	remaining: 3.48s
351:	learn: 0.3642771	total: 1.89s	remaining: 3.48s
352:	learn: 0.3640861	total: 1.9s	remaining: 3.47s
353:	learn: 0.3638501	total: 1.9s	remaining: 3.47s
354:	learn: 0.3635260	total: 1.91s	remaining: 3.47s
355:	learn: 0.3631299	total: 1.92s	remaining: 3.46s
356:	learn: 0.3629424	total: 1.92s	remaining: 3.46s
357:	learn: 0.3623963	total: 1.93s	remaining: 3.46s
358:	learn: 0.3620717	total: 1.94s	remaining: 3.46s
359:	learn: 0.3617857	total: 1.94s	remaining: 3.45s
360:	learn: 0.3616258	total: 1.95s	remaining: 3.45s
361:	learn: 0.3612248	total: 1.95s	remaining: 3.44s
362:	learn: 0.3608925	total: 1.96s	remaining: 3.44s
363:	learn: 0.3605049	total: 1.97s	remaining: 3.44s
364:	learn: 0.3603122	total: 1.97s	remaining: 3.43s
365:	learn: 0.3599961	total: 1.98s	remaining: 3.43s
366:	learn: 0.3595355	total: 1.99s	remaining: 3.42s
367:	learn: 0.

530:	learn: 0.3223724	total: 2.68s	remaining: 2.36s
531:	learn: 0.3222371	total: 2.68s	remaining: 2.36s
532:	learn: 0.3218400	total: 2.69s	remaining: 2.36s
533:	learn: 0.3216354	total: 2.7s	remaining: 2.36s
534:	learn: 0.3215363	total: 2.71s	remaining: 2.35s
535:	learn: 0.3213862	total: 2.71s	remaining: 2.35s
536:	learn: 0.3213006	total: 2.72s	remaining: 2.34s
537:	learn: 0.3210591	total: 2.73s	remaining: 2.34s
538:	learn: 0.3207618	total: 2.73s	remaining: 2.34s
539:	learn: 0.3205750	total: 2.74s	remaining: 2.34s
540:	learn: 0.3204209	total: 2.75s	remaining: 2.33s
541:	learn: 0.3201659	total: 2.76s	remaining: 2.33s
542:	learn: 0.3198738	total: 2.77s	remaining: 2.33s
543:	learn: 0.3196935	total: 2.77s	remaining: 2.32s
544:	learn: 0.3195247	total: 2.78s	remaining: 2.32s
545:	learn: 0.3193739	total: 2.79s	remaining: 2.32s
546:	learn: 0.3191856	total: 2.79s	remaining: 2.31s
547:	learn: 0.3190313	total: 2.8s	remaining: 2.31s
548:	learn: 0.3188659	total: 2.81s	remaining: 2.31s
549:	learn: 0.

717:	learn: 0.2873668	total: 3.85s	remaining: 1.51s
718:	learn: 0.2872943	total: 3.85s	remaining: 1.51s
719:	learn: 0.2872110	total: 3.86s	remaining: 1.5s
720:	learn: 0.2870868	total: 3.87s	remaining: 1.5s
721:	learn: 0.2867275	total: 3.88s	remaining: 1.49s
722:	learn: 0.2865065	total: 3.88s	remaining: 1.49s
723:	learn: 0.2863663	total: 3.89s	remaining: 1.48s
724:	learn: 0.2861463	total: 3.9s	remaining: 1.48s
725:	learn: 0.2859523	total: 3.91s	remaining: 1.47s
726:	learn: 0.2858347	total: 3.91s	remaining: 1.47s
727:	learn: 0.2856052	total: 3.92s	remaining: 1.46s
728:	learn: 0.2853423	total: 3.93s	remaining: 1.46s
729:	learn: 0.2850713	total: 3.93s	remaining: 1.46s
730:	learn: 0.2849300	total: 3.94s	remaining: 1.45s
731:	learn: 0.2848137	total: 3.95s	remaining: 1.45s
732:	learn: 0.2847105	total: 3.96s	remaining: 1.44s
733:	learn: 0.2846858	total: 3.96s	remaining: 1.44s
734:	learn: 0.2845458	total: 3.97s	remaining: 1.43s
735:	learn: 0.2843515	total: 3.98s	remaining: 1.43s
736:	learn: 0.2

882:	learn: 0.2595809	total: 4.7s	remaining: 623ms
883:	learn: 0.2594020	total: 4.71s	remaining: 617ms
884:	learn: 0.2592742	total: 4.71s	remaining: 612ms
885:	learn: 0.2591341	total: 4.71s	remaining: 606ms
886:	learn: 0.2590314	total: 4.71s	remaining: 601ms
887:	learn: 0.2589807	total: 4.72s	remaining: 595ms
888:	learn: 0.2587908	total: 4.72s	remaining: 589ms
889:	learn: 0.2585689	total: 4.72s	remaining: 584ms
890:	learn: 0.2583623	total: 4.73s	remaining: 578ms
891:	learn: 0.2582002	total: 4.73s	remaining: 573ms
892:	learn: 0.2580934	total: 4.73s	remaining: 567ms
893:	learn: 0.2579772	total: 4.74s	remaining: 562ms
894:	learn: 0.2577315	total: 4.74s	remaining: 556ms
895:	learn: 0.2575329	total: 4.74s	remaining: 551ms
896:	learn: 0.2573933	total: 4.75s	remaining: 545ms
897:	learn: 0.2572588	total: 4.75s	remaining: 540ms
898:	learn: 0.2571714	total: 4.75s	remaining: 534ms
899:	learn: 0.2569795	total: 4.76s	remaining: 529ms
900:	learn: 0.2568192	total: 4.76s	remaining: 523ms
901:	learn: 0

In [24]:
print(f"Logistic Regression score = {lr_score.mean()}")
print(f"SVC Regression score = {svc_score.mean()}")
print(f"Random Forest Regression score = {rf_score.mean()}")
print(f"AdaBost Regression score = {ada_score.mean()}")
print(f"XGB Regression score = {xgb_score.mean()}")
print(f"CatBoost Regression score = {cat_score.mean()}")

Logistic Regression score = 0.7915367186458749
SVC Regression score = 0.7996801279488205
Random Forest Regression score = 0.7801812608290019
AdaBost Regression score = 0.789870718379315
XGB Regression score = 0.7507930161268825
CatBoost Regression score = 0.7996801279488205


#### Both have given almost same results but Knn imputation is almost 10 times faster than Iterative Imputer

In [25]:
#Moving Forward with KNN imputer results
X_train_imputed = pd.DataFrame(knn_imputed, columns = X_train.columns)

In [26]:
X_train_imputed.head()

,Gender,Married,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Credit_History,Total_Income,Semiurban,Urban
0,1.0,0.0,0.0,1.0,0.0,132.2,360.0,1.0,5849.0,0.0,1.0
1,1.0,1.0,1.0,1.0,0.0,128.0,360.0,1.0,6091.0,0.0,0.0
2,1.0,1.0,0.0,1.0,1.0,66.0,360.0,1.0,3000.0,0.0,1.0
3,1.0,1.0,0.0,0.0,0.0,120.0,360.0,1.0,4941.0,0.0,1.0
4,1.0,0.0,0.0,1.0,0.0,141.0,360.0,1.0,6000.0,0.0,1.0


In [27]:
train_df_imputed = X_train_imputed

In [28]:
train_df_imputed.insert(0,"Loan_ID", value=train_df["Loan_ID"])

In [29]:
train_df_imputed.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Credit_History,Total_Income,Semiurban,Urban
0,LP001002,1.0,0.0,0.0,1.0,0.0,132.2,360.0,1.0,5849.0,0.0,1.0
1,LP001003,1.0,1.0,1.0,1.0,0.0,128.0,360.0,1.0,6091.0,0.0,0.0
2,LP001005,1.0,1.0,0.0,1.0,1.0,66.0,360.0,1.0,3000.0,0.0,1.0
3,LP001006,1.0,1.0,0.0,0.0,0.0,120.0,360.0,1.0,4941.0,0.0,1.0
4,LP001008,1.0,0.0,0.0,1.0,0.0,141.0,360.0,1.0,6000.0,0.0,1.0


In [30]:
train_df_imputed["Loan_Status"] = train_df["Loan_Status"]

In [31]:
train_df.shape

(614, 13)

In [32]:
check_null(train_df_imputed)

Loan_ID             0.0
Gender              0.0
Married             0.0
Dependents          0.0
Education           0.0
Self_Employed       0.0
LoanAmount          0.0
Loan_Amount_Term    0.0
Credit_History      0.0
Total_Income        0.0
Semiurban           0.0
Urban               0.0
Loan_Status         0.0
dtype: float64

In [33]:
#saving the imputed data
train_df_imputed.to_csv("../data/train_imputed.csv", index=False)

## imputing the test csv file

In [34]:
test_df = pd.read_csv("../data/test_encoded.csv")

In [35]:
test_df.shape

(367, 12)

In [36]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Loan_ID           367 non-null    object 
 1   Gender            356 non-null    float64
 2   Married           367 non-null    int64  
 3   Dependents        357 non-null    float64
 4   Education         367 non-null    int64  
 5   Self_Employed     344 non-null    float64
 6   LoanAmount        362 non-null    float64
 7   Loan_Amount_Term  361 non-null    float64
 8   Credit_History    338 non-null    float64
 9   Semiurban         367 non-null    int64  
 10  Urban             367 non-null    int64  
 11  Total_Income      367 non-null    int64  
dtypes: float64(6), int64(5), object(1)
memory usage: 34.5+ KB


In [37]:
test_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Credit_History,Semiurban,Urban,Total_Income
0,LP001015,1.0,1,0.0,1,0.0,110.0,360.0,1.0,0,1,5720
1,LP001022,1.0,1,1.0,1,0.0,126.0,360.0,1.0,0,1,4576
2,LP001031,1.0,1,2.0,1,0.0,208.0,360.0,1.0,0,1,6800
3,LP001035,1.0,1,2.0,1,0.0,100.0,360.0,NaN,0,1,4886
4,LP001051,1.0,0,0.0,0,0.0,78.0,360.0,1.0,0,1,3276


In [38]:
check_null(test_df)

Credit_History      7.901907
Self_Employed       6.267030
Gender              2.997275
Dependents          2.724796
Loan_Amount_Term    1.634877
LoanAmount          1.362398
Loan_ID             0.000000
Married             0.000000
Education           0.000000
Semiurban           0.000000
Urban               0.000000
Total_Income        0.000000
dtype: float64

In [39]:
X_test = test_df.drop("Loan_ID", axis=1)

In [40]:
X_test_imputed = knn_imputer.transform(X_test)

In [41]:
test_df_imputed = pd.DataFrame(X_test_imputed, columns=X_test.columns)

In [42]:
test_df_imputed.insert(0, column="Loan_ID", value=test_df["Loan_ID"])

In [43]:
test_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Credit_History,Semiurban,Urban,Total_Income
0,LP001015,1.0,1,0.0,1,0.0,110.0,360.0,1.0,0,1,5720
1,LP001022,1.0,1,1.0,1,0.0,126.0,360.0,1.0,0,1,4576
2,LP001031,1.0,1,2.0,1,0.0,208.0,360.0,1.0,0,1,6800
3,LP001035,1.0,1,2.0,1,0.0,100.0,360.0,NaN,0,1,4886
4,LP001051,1.0,0,0.0,0,0.0,78.0,360.0,1.0,0,1,3276


In [44]:
check_null(test_df_imputed)

Loan_ID             0.0
Gender              0.0
Married             0.0
Dependents          0.0
Education           0.0
Self_Employed       0.0
LoanAmount          0.0
Loan_Amount_Term    0.0
Credit_History      0.0
Semiurban           0.0
Urban               0.0
Total_Income        0.0
dtype: float64

In [45]:
#saving the test file
test_df_imputed.to_csv("../data/test_imputed.csv", index=False)